## LDA

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

## Metrics

In [ ]:
def rocauc(true, score):
    # sort indices by probability
    sorted_indices = np.argsort(score)

    # sort the actuals by probability (l to g)
    true_sort = true[sorted_indices]

    # sort the probability (l to g)
    score_sort = score[sorted_indices]

    # vector of true positives (cumulative sum)
    TP = np.cumsum(true_sort)

    # vector of false positives (cumulative sum)
    FP = np.cumsum(1 - true_sort)

    # get true positive rate vector
    TPR = TP / TP[-1]

    # get false positive rate vector
    FPR = FP / FP[-1]

    # area under curve (TPR vs. FPR)
    rocauc_score = np.trapz(FPR, TPR)

    return rocauc_score

def metrics(true, pred):
    # get TP, TN, FP, FN
    TP = np.sum((pred == 1) & (true == 1))
    TN = np.sum((pred == 0) & (true == 0))
    FP = np.sum((pred == 1) & (true == 0))
    FN = np.sum((pred == 0) & (true == 1))

    # accuracy
    accuracy = (TP + TN) / (TP + TN + FP + FN)

    # precision
    if TP + FP > 0:
        precision = TP / (TP + FP)
    else:
        precision = 0

    # recall
    if TP + FN > 0:
        recall = TP / (TP + FN)
    else:
        recall = 0

    # f1 score
    f1 = 0
    if precision + recall > 0:
        f1 = 2 * precision * recall / (precision + recall)
    else:
        recall = 0

    return accuracy, precision, recall, f1

def matthews_correlation_coefficient(y_true, y_pred):
    tp = sum((y_true == 1) & (y_pred == 1))
    tn = sum((y_true == 0) & (y_pred == 0))
    fp = sum((y_true == 0) & (y_pred == 1))
    fn = sum((y_true == 1) & (y_pred == 0))

    numerator = (tp * tn) - (fp * fn)
    denominator = ((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn)) ** 0.5

    if denominator == 0:
        return 0
    return numerator / denominator

## Splits

In [ ]:
def random_split(X, y, train_ratio=0.6, val_ratio=0.2):
    n_total = X.shape[0]
    n_train = int(n_total * train_ratio)
    n_val = int(n_total * val_ratio)

    indices = np.random.permutation(n_total)

    train_indices = indices[:n_train]
    val_indices = indices[n_train:n_train + n_val]
    test_indices = indices[n_train + n_val:]

    X_train, y_train = X.iloc[train_indices], y.iloc[train_indices]
    X_val, y_val = X.iloc[val_indices], y.iloc[val_indices]
    X_test, y_test = X.iloc[test_indices], y.iloc[test_indices]

    return X_train, y_train, X_val, y_val, X_test, y_test

def stratified_split(X, y, train_ratio=0.6, val_ratio=0.2):
    unique_classes, class_counts = np.unique(y, return_counts=True)
    train_indices, val_indices, test_indices = [], [], []

    for cls, count in zip(unique_classes, class_counts):
        cls_indices = np.where(y == cls)[0]
        np.random.shuffle(cls_indices)
        n_train_cls = int(count * train_ratio)
        n_val_cls = int(count * val_ratio)

        train_indices.extend(cls_indices[:n_train_cls])
        val_indices.extend(cls_indices[n_train_cls:n_train_cls + n_val_cls])
        test_indices.extend(cls_indices[n_train_cls + n_val_cls:])

    train_indices = np.array(train_indices)
    val_indices = np.array(val_indices)
    test_indices = np.array(test_indices)

    np.random.shuffle(train_indices)
    np.random.shuffle(val_indices)
    np.random.shuffle(test_indices)

    X_train, y_train = X.iloc[train_indices], y.iloc[train_indices]
    X_val, y_val = X.iloc[val_indices], y.iloc[val_indices]
    X_test, y_test = X.iloc[test_indices], y.iloc[test_indices]

    return X_train, y_train, X_val, y_val, X_test, y_test

def sorted_split(X, y, sort_column, train_ratio=0.6, val_ratio=0.2):
    sorted_indices = X[sort_column].argsort()
    X_sorted = X.iloc[sorted_indices]
    y_sorted = y.iloc[sorted_indices]

    n_total = X.shape[0]
    n_train = int(n_total * train_ratio)
    n_val = int(n_total * val_ratio)
    n_test = n_total - n_train - n_val

    train_indices, val_indices, test_indices = [], [], []
    block_size = 10

    for i in range(0, n_total, block_size):
        block_indices = range(i, min(i + block_size, n_total))
        if len(train_indices) < n_train:
            train_indices.extend(block_indices)
        elif len(val_indices) < n_val:
            val_indices.extend(block_indices)
        else:
            test_indices.extend(block_indices)

    X_train, y_train = X_sorted.iloc[train_indices], y_sorted.iloc[train_indices]
    X_val, y_val = X_sorted.iloc[val_indices], y_sorted.iloc[val_indices]
    X_test, y_test = X_sorted.iloc[test_indices], y_sorted.iloc[test_indices]

    return X_train, y_train, X_val, y_val, X_test, y_test

## DATA

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
df_merged = pd.read_csv('/content/drive/MyDrive/DATA 403 Project 2/merge_cleaned.csv')
df_application = pd.read_csv('/content/drive/MyDrive/DATA 403 Project 2/application_train_cleaned.csv')

columns_to_scale = ['AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']

preprocessor = ColumnTransformer(
    transformers=[
        ('scale', StandardScaler(), columns_to_scale)
    ], remainder='passthrough'
)

df_mergedX = df_merged.drop(columns=['TARGET'])
df_mergedy = df_merged['TARGET']

df_applicationX = df_application.drop(columns=['TARGET'])
df_applicationy = df_application['TARGET']


Mounted at /content/drive


## Random Split

In [ ]:
X_train, y_train, X_val, y_val, X_test, y_test = random_split(df_mergedX, df_mergedy)
y_test = y_test.to_numpy()

X_train = preprocessor.fit_transform(X_train)
X_val = preprocessor.transform(X_val)
X_test = preprocessor.transform(X_test)

model = LinearDiscriminantAnalysis()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
y_prob = model.decision_function(X_test)

accuracy, precision, recall, f1_score = metrics(y_test, y_pred)
mcc = matthews_correlation_coefficient(y_test, y_pred)
rocauc_score = rocauc(y_test, y_prob)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1_score}')
print(f'MCC: {mcc}')
print(f'ROC AUC: {rocauc_score}')

# metrics on validation set:
y_val_pred = model.predict(X_val)
y_val_prob = model.decision_function(X_val)
val_accuracy, val_precision, val_recall, val_f1_score = metrics(y_val, y_val_pred)
val_mcc = matthews_correlation_coefficient(y_val, y_val_pred)
val_rocauc_score = rocauc(y_val, y_val_prob)

print(f'Validation Accuracy: {val_accuracy}')
print(f'Validation Precision: {val_precision}')
print(f'Validation Recall: {val_recall}')
print(f'Validation F1 Score: {val_f1_score}')
print(f'Validation MCC: {val_mcc}')
print(f'Validation ROC AUC: {val_rocauc_score}')

Accuracy: 0.9237795131753208
Precision: 0.5238095238095238
Recall: 0.02630005977286312
F1 Score: 0.05008537279453615
MCC: 0.10451706842659433
ROC AUC: 0.7496943379398792


In [ ]:
X_train, y_train, X_val, y_val, X_test, y_test = random_split(df_applicationX, df_applicationy)
y_test = y_test.to_numpy()

X_train = preprocessor.fit_transform(X_train)
X_val = preprocessor.transform(X_val)
X_test = preprocessor.transform(X_test)

model = LinearDiscriminantAnalysis()
model.fit(X_train, y_train)


y_pred = model.predict(X_test)
y_prob = model.decision_function(X_test)

accuracy, precision, recall, f1_score = metrics(y_test, y_pred)
mcc = matthews_correlation_coefficient(y_test, y_pred)
rocauc_score = rocauc(y_test, y_prob)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1_score}')
print(f'MCC: {mcc}')
print(f'ROC AUC: {rocauc_score}')

# metrics on validation set:
y_val_pred = model.predict(X_val)
y_val_prob = model.decision_function(X_val)
val_accuracy, val_precision, val_recall, val_f1_score = metrics(y_val, y_val_pred)
val_mcc = matthews_correlation_coefficient(y_val, y_val_pred)
val_rocauc_score = rocauc(y_val, y_val_prob)

print(f'Validation Accuracy: {val_accuracy}')
print(f'Validation Precision: {val_precision}')
print(f'Validation Recall: {val_recall}')
print(f'Validation F1 Score: {val_f1_score}')
print(f'Validation MCC: {val_mcc}')
print(f'Validation ROC AUC: {val_rocauc_score}')

Accuracy: 0.9267479563410513
Precision: 0.5079365079365079
Recall: 0.019937694704049845
F1 Score: 0.03836930455635492
MCC: 0.08958118102516177
ROC AUC: 0.7583354979910135


## Stratified Split

In [ ]:
X_train, y_train, X_val, y_val, X_test, y_test = stratified_split(df_mergedX, df_mergedy)
y_test = y_test.to_numpy()

X_train = preprocessor.fit_transform(X_train)
X_val = preprocessor.transform(X_val)
X_test = preprocessor.transform(X_test)

model = LinearDiscriminantAnalysis()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
y_prob = model.decision_function(X_test)

accuracy, precision, recall, f1_score = metrics(y_test, y_pred)
mcc = matthews_correlation_coefficient(y_test, y_pred)
rocauc_score = rocauc(y_test, y_prob)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1_score}')
print(f'MCC: {mcc}')
print(f'ROC AUC: {rocauc_score}')

# metrics on validation set:
y_val_pred = model.predict(X_val)
y_val_prob = model.decision_function(X_val)
val_accuracy, val_precision, val_recall, val_f1_score = metrics(y_val, y_val_pred)
val_mcc = matthews_correlation_coefficient(y_val, y_val_pred)
val_rocauc_score = rocauc(y_val, y_val_prob)

print(f'Validation Accuracy: {val_accuracy}')
print(f'Validation Precision: {val_precision}')
print(f'Validation Recall: {val_recall}')
print(f'Validation F1 Score: {val_f1_score}')
print(f'Validation MCC: {val_mcc}')
print(f'Validation ROC AUC: {val_rocauc_score}')

Accuracy: 0.9269339665722897
Precision: 0.5
Recall: 0.021875
F1 Score: 0.041916167664670656
MCC: 0.09290104342214624
ROC AUC: 0.755445548822544


In [ ]:
X_train, y_train, X_val, y_val, X_test, y_test = stratified_split(df_applicationX, df_applicationy)
y_test = y_test.to_numpy()

X_train = preprocessor.fit_transform(X_train)
X_val = preprocessor.transform(X_val)
X_test = preprocessor.transform(X_test)

model = LinearDiscriminantAnalysis()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
y_prob = model.decision_function(X_test)

accuracy, precision, recall, f1_score = metrics(y_test, y_pred)
mcc = matthews_correlation_coefficient(y_test, y_pred)
rocauc_score = rocauc(y_test, y_prob)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1_score}')
print(f'MCC: {mcc}')
print(f'ROC AUC: {rocauc_score}')

# metrics on validation set:
y_val_pred = model.predict(X_val)
y_val_prob = model.decision_function(X_val)
val_accuracy, val_precision, val_recall, val_f1_score = metrics(y_val, y_val_pred)
val_mcc = matthews_correlation_coefficient(y_val, y_val_pred)
val_rocauc_score = rocauc(y_val, y_val_prob)

print(f'Validation Accuracy: {val_accuracy}')
print(f'Validation Precision: {val_precision}')
print(f'Validation Recall: {val_recall}')
print(f'Validation F1 Score: {val_f1_score}')
print(f'Validation MCC: {val_mcc}')
print(f'Validation ROC AUC: {val_rocauc_score}')

Accuracy: 0.9272992967394282
Precision: 0.5571428571428572
Recall: 0.024375
F1 Score: 0.046706586826347304
MCC: 0.10533535755113274
ROC AUC: 0.7571320142378559


## Ordered Split

In [ ]:
X_train, y_train, X_val, y_val, X_test, y_test = sorted_split(df_mergedX, df_mergedy, 'AMT_INCOME_TOTAL')
y_test = y_test.to_numpy()

X_train = preprocessor.fit_transform(X_train)
X_val = preprocessor.transform(X_val)
X_test = preprocessor.transform(X_test)

model = LinearDiscriminantAnalysis()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
y_prob = model.decision_function(X_test)

accuracy, precision, recall, f1_score = metrics(y_test, y_pred)
mcc = matthews_correlation_coefficient(y_test, y_pred)
rocauc_score = rocauc(y_test, y_prob)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1_score}')
print(f'MCC: {mcc}')
print(f'ROC AUC: {rocauc_score}')

# metrics on validation set:
y_val_pred = model.predict(X_val)
y_val_prob = model.decision_function(X_val)
val_accuracy, val_precision, val_recall, val_f1_score = metrics(y_val, y_val_pred)
val_mcc = matthews_correlation_coefficient(y_val, y_val_pred)
val_rocauc_score = rocauc(y_val, y_val_prob)

print(f'Validation Accuracy: {val_accuracy}')
print(f'Validation Precision: {val_precision}')
print(f'Validation Recall: {val_recall}')
print(f'Validation F1 Score: {val_f1_score}')
print(f'Validation MCC: {val_mcc}')
print(f'Validation ROC AUC: {val_rocauc_score}')

Accuracy: 0.9395185235941711
Precision: 0.43243243243243246
Recall: 0.012130401819560273
F1 Score: 0.02359882005899705
MCC: 0.06435653053805691
ROC AUC: 0.7616315897551409


In [ ]:
X_train, y_train, X_val, y_val, X_test, y_test = sorted_split(df_applicationX, df_applicationy, 'AMT_INCOME_TOTAL')
y_test = y_test.to_numpy()

X_train = preprocessor.fit_transform(X_train)
X_val = preprocessor.transform(X_val)
X_test = preprocessor.transform(X_test)

model = LinearDiscriminantAnalysis()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
y_prob = model.decision_function(X_test)

accuracy, precision, recall, f1_score = metrics(y_test, y_pred)
mcc = matthews_correlation_coefficient(y_test, y_pred)
rocauc_score = rocauc(y_test, y_prob)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1_score}')
print(f'MCC: {mcc}')
print(f'ROC AUC: {rocauc_score}')

# metrics on validation set:
y_val_pred = model.predict(X_val)
y_val_prob = model.decision_function(X_val)
val_accuracy, val_precision, val_recall, val_f1_score = metrics(y_val, y_val_pred)
val_mcc = matthews_correlation_coefficient(y_val, y_val_pred)
val_rocauc_score = rocauc(y_val, y_val_prob)

print(f'Validation Accuracy: {val_accuracy}')
print(f'Validation Precision: {val_precision}')
print(f'Validation Recall: {val_recall}')
print(f'Validation F1 Score: {val_f1_score}')
print(f'Validation MCC: {val_mcc}')
print(f'Validation ROC AUC: {val_rocauc_score}')

Accuracy: 0.9397012470878443
Precision: 0.4878048780487805
Recall: 0.015163002274450341
F1 Score: 0.029411764705882356
MCC: 0.07783230609274376
ROC AUC: 0.7592133002202217
